<hr style="border:2px solid #808080"> </hr>
<center><h1 style="color:#03122E;"> Álgebra Lineal Numérica IMT2111</h1></center>
<center><h1 style="color:#173F8A;"> Capítulo 1: Eliminacion Gaussiana</h3></center>
<center><h1 style="color:#0176DE;"> Prof. Manuel A. Sánchez</h3></center>
<hr style="border:2px solid #808080"> </hr>

## Tabla de contenidos
1. Eliminacion Gaussiana
2. Ejemplo Worst-case instability

In [35]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
display(HTML("""<style>.output {display: flex;align-items: center;text-align: center;}</style>"""))

In [188]:
# Partial pivoting
def GEPP(Ainput, binput):
    A = Ainput.copy()
    b = binput.copy()
    print(b)
    '''
    Gaussian elimination with partial (row) pivoting
    input : A nonsingular and square matrix n x n 
            b vector n x 1
    output : x solution of the system A x = b
    '''
    # 1. Factorize A = PLU
    L, U, pT = LUPP(A)
    print("U:\n", U)
    # 2. Solve P L U x = b
#     Ptb = b[pT]
#     Ptb = permb(b, pT)
    P = perm(pT)
#     print(P)
#     print("P^T  *b = ", Ptb)
#     print(P, b)
    rhs  = perm(pT).dot(b)
#     print("P^T  *b = ", Ptb)
#     print(L)

    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return x

def LUPP(Ainput):
    '''
    LU factorization with partial pivoting
    '''
    A = Ainput.copy()
    n = A.shape[0]
    piv = np.arange(0,n-1)
    for i in range(n-1):
        imax = abs(A[i:,i]).argmax() + i
        piv[i] = imax
        if A[imax, i] == 0:
            raise ValueError("Matrix is singular.")
        elif imax != i:
            A[[i,imax],:] = A[[imax, i],:][:]
        A[(i+1):n,i][:] = (A[(i+1):n,i]/A[i,i])[:]
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, piv

# Complete pivoting
def GECP(Ainput, binput):
    '''
    Gaussian elimination with complete pivoting
    input : A nonsingular and square matrix n x n 
            b vector n x 1
    output : x solution of the system A x = b
    '''
    A = Ainput.copy()
    b = binput.copy()
    # 1. Factorize PAQ^T = LU
    L, U, rowpiv, colpiv = LUCP(A)
    print("U:\n", U)
    # 2. Solve  L U x = P b Q^T
#     for k in range(b.size-1):
#         b[[k,rowpiv[k]]] = b[[rowpiv[k], k]][:]
    rhs  = perm(rowpiv).dot(b)
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, rhs)
    
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    print("GECP", x)
#     for k in range(x.size-1):
#          x[[colpiv[k], k]] =x[[k,colpiv[k]]]
#     print(x)
    return (perm(colpiv).T).dot(x)

def LUCP(Ainput):
    '''
    LU factorization with complete pivoting
    '''
    A = Ainput.copy()
    n = A.shape[0]
    rowpiv = np.arange(0,n-1)
    colpiv = np.arange(0,n-1)
    for i in range(n-1):
        mu, lam = np.unravel_index(np.argmax(A[i:,i:], axis=None), A[i:,i:].shape)
        mu +=i; lam+=i
        if A[mu, lam] == 0:
            raise ValueError("Matrix is singular.")
        else:
            #print("A before complete pivoting")
            #print(A)
            rowpiv[i] = mu
            A[[i, mu],:] = A[[mu, i],:][:]
            colpiv[i] = lam
            A[:,[i, lam]] = A[:,[lam, i]][:]
            #print("A after complete pivoting")
            #print(A)
        #print(A[i+1,i],A[(i+1):n,i])
        
        A[(i+1):n,i] *= 1.0/A[i,i]
        #print(A[(i+1):n,i] ,  (A[(i+1):n,i]/A[i,i]))
        
        A[(i+1):n, (i+1):n] -=np.outer(A[(i+1):n,i],A[i, (i+1):n])
        #print("new A")
        #print(A)
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, rowpiv, colpiv


# generate permutation matrix
def perm(rowpiv):
    n = rowpiv.size+1
    print(rowpiv)
    P = np.eye(n)
    for k in range(n-1):
        P[[k,rowpiv[k]],:] = P[[rowpiv[k], k],:]
    return P
def permb(b, rowpiv):
    n = b.size
    for k in range(n-1):
        b[[k, rowpiv[k]]] = b[[rowpiv[k],k]] 
    return b
# No pivoting
def GE(Ainput, binput):
    '''
    Gaussian elimination without pivoting
    input : A nonsingular and square matrix n x n 
            b vector n x 1
    output : x solution of the system A x = b
    '''
    A = Ainput.copy()
    b= binput.copy()
    # 1. Factorize A = LU
    L, U = LU(A)
    # 2. Solve LUx = b forward substitution
    y = forward_substitution(L, b)
    # 3. Solve Ux = L^{-1} b backward substitution
    x = backward_substitution(U,y)
    return x


def LU(Ainput):
    '''
    LU factorization without pivoting
    '''
    A = Ainput.copy()
    n = A.shape[0]
    for i in range(n-1):
        if A[i, i] == 0:
            raise ValueError("coeficient is zero.")
        A[(i+1):n,i] = (A[(i+1):n,i]/A[i,i])
        
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U


# Forward and Backward substitution
def forward_substitution(L,b):
    '''
    Forward substitution algorithm for system L x = b
    input : L lower triangular matrix n x n
            b vector n x 1
    output: x solution of L x = b
    '''
    n = L.shape[0]
    x = np.zeros(n)
    x[0] = b[0]/L[0,0]
    for i in range(1,n):
        x[i] = (b[i] - L[i,0:i]@x[0:i])/L[i,i]
    return x

def backward_substitution(U,b):
    '''
    Backward substitution algorithm for system U x = b
    input : U upper tringular matrix n x n
            b vector n x 1
    output : x solution of U x = b
    '''
    n = U.shape[0]
    x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - U[i,(i+1):n]@(x[(i+1):n]))/U[i,i]
    return x



In [189]:
def Eliminacion_Gaussiana(A,b, pivoteo=None):
    if pivoteo is None:
        x = GE(A,b)
    elif pivoteo == 'parcial':
        x = GEPP(A,b)
    elif pivoteo == 'completo':
        x = GECP(A,b)
    return x

def Factorizacion_LU(A, pivoteo=None):
    if pivoteo is None:
        L,U = LU(A)
        return L,U
    elif pivoteo == 'parcial':
        L,U,piv = LUPP(A)
        return L, U, piv
    elif pivoteo == 'completo':
        L, U, rowpiv, colpiv = LUCP(A)
        return  L, U, rowpiv, colpiv

## Ejemplo: eliminacion Gaussiana
\begin{equation}
Ax = b,\qquad 
A = \begin{bmatrix}
1 & 3 & 4 & 1\\
2 & 1 & 5 & 1 \\
3 & 1 & 6 & 1 \\
6 & 2 & 3 & 2
\end{bmatrix},\quad
b = \begin{bmatrix}
-1 \\ 2\\ 3 \\6
\end{bmatrix}
\end{equation}

In [190]:
A = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
b = np.array([-1,2,3,6],dtype=np.float64)


In [191]:
x = Eliminacion_Gaussiana(A,b)
# x= np.array([1,-1,0,1])
print(x)

[ 1. -1. -0.  1.]


In [192]:
L,U = Factorizacion_LU(A)
print(L)
print(U)
print(L@U-A)

[[1.  0.  0.  0. ]
 [2.  1.  0.  0. ]
 [3.  1.6 1.  0. ]
 [6.  3.2 9.5 1. ]]
[[ 1.   3.   4.   1. ]
 [ 0.  -5.  -3.  -1. ]
 [ 0.   0.  -1.2 -0.4]
 [ 0.   0.   0.   3. ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -4.44089210e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -8.88178420e-16 -8.88178420e-16 -2.22044605e-16]]


## Ejemplo: Factorizacion LU
\begin{equation}
LU = A,\qquad 
A = \begin{bmatrix}
2 & 1 & 1 & 0\\
4 & 3 & 3 & 1 \\
8 & 7 & 9 & 5 \\
6 & 7 & 9 & 8
\end{bmatrix}
\end{equation}

In [193]:
A2 = np.array([[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]],dtype=np.float64)

In [194]:
L,U = Factorizacion_LU(A2)
print(L)
print(U)
print(L@U-A)

[[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [4. 3. 1. 0.]
 [3. 4. 1. 1.]]
[[2. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]]
[[ 1. -2. -3. -1.]
 [ 2.  2. -2.  0.]
 [ 5.  6.  3.  4.]
 [ 0.  5.  6.  6.]]


## Ejemplo: inestabilidad sin pivoteo
\begin{equation}
A = \begin{bmatrix}
0 & 1 \\
1 & 1
\end{bmatrix}, \quad \text{Eliminacion Gaussiana falla en el primer paso con } \kappa(A) = (3+\sqrt{5})/2
\end{equation}

\begin{equation}
\tilde{A} = \begin{bmatrix}
10^{-20} & 1 \\
1 & 1
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 \\
10^{20} & 1
\end{bmatrix}
\begin{bmatrix}
10^{-20} & 1 \\
0 & 1-10^{20}
\end{bmatrix}, \quad \text{Eliminacion Gaussiana ahora no falla}
\end{equation}

In [195]:
Afalla = np.array([[0,1],[1,1]], dtype=np.float64)
Atilde = np.array([[10**(-20), 1],[1,1]], dtype=np.float64)

In [196]:
L,U = Factorizacion_LU(Afalla)


ValueError: coeficient is zero.

In [197]:
L,U = Factorizacion_LU(Atilde)
print(L)
print(U)
print(L@U- Atilde)

[[1.e+00 0.e+00]
 [1.e+20 1.e+00]]
[[ 1.e-20  1.e+00]
 [ 0.e+00 -1.e+20]]
[[ 0.  0.]
 [ 0. -1.]]


In [198]:
b = np.array([1,0])
xtilde = Eliminacion_Gaussiana(Atilde,b)
print("solucion calculada:", xtilde)
print("la solucion exacta es x = [-1,1]!")

solucion calculada: [0. 1.]
la solucion exacta es x = [-1,1]!


**Nota** La factorizacion LU es estable, no backward stable. Pero al usar eliminacion Gaussiana sin pivoteo para resolver $Ax=b$ no lo hace estable.

En general, si un paso del algoritmo es estable pero backward stable para resolver un subproblema, entonces la estabilidad del algortimo puede estar en peligro.

**Nota** Complejidad:  eliminacion Gaussiana sin pivoteo es de $\approx \frac{2}{3} m^{3}$ flops.

## Ejemplo: worst case

## Inestabilidad

Para ciertas matrices $A$ a pesar de os efectos beneficiosos de pivotear, el factor de crecimiento $\rho$ se vuelve gigante. Por ejemplo, suponga que la matriz $A$ tiene la siguiente forma:

$$
A = \begin{bmatrix}
1 & 0 & 0 & 0 & 1 \\
-1 &1 &0 & 0& 1 \\
-1& -1 &1 & 0 &1\\
-1&-1 &-1 & 1 & 1 \\
-1& -1 &-1 &-1 &1
\end{bmatrix}
$$

En este caso, la factorization da 

$$
U = \begin{bmatrix}
 1. & 0. & 0. & 0.&  1.\\
 0. & 1. & 0. & 0.&  2.\\
 0. & 0. & 1. & 0.&  4.\\
 0. & 0. & 0. & 1.&  8.\\
 0. & 0. & 0. & 0.& 16.
 \end{bmatrix}
$$

para esta matriz de $n\times n$, con $n=5$, el factor de crecimiento es $\rho = 2^{n-1} = 16$.

Un factor de crecimiento de orden $2^{n}$ corresponde a una perdida del orden de $n$ bits de precision, lo cual es catastrofico para computaciones practicas. Como un computador standard representa numeros de punto flotante con solo 64 bits, con matries de dimensiones en cientos y miles en dimension perder m bits de precision es intolerable.

In [199]:
n = 5
A_wc = -np.tril(np.ones((n,n)),-1)+np.eye(n)
A_wc[:,-1] = np.ones(n)
A_wc

array([[ 1.,  0.,  0.,  0.,  1.],
       [-1.,  1.,  0.,  0.,  1.],
       [-1., -1.,  1.,  0.,  1.],
       [-1., -1., -1.,  1.,  1.],
       [-1., -1., -1., -1.,  1.]])

In [200]:
L, U, pT = Factorizacion_LU(A_wc, pivoteo='parcial')
U

array([[ 1.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  2.],
       [ 0.,  0.,  1.,  0.,  4.],
       [ 0.,  0.,  0.,  1.,  8.],
       [ 0.,  0.,  0.,  0., 16.]])

# Example. 
Consider the system $Ax = b$ with 
\begin{equation}
A = 
\begin{pmatrix}
6 & 3 & 4 & 1 \\ 2 & 1 & 5 & 1 \\ 1 & 1 & 6 & 1 \\ 0 & 2 & 3 & 2
\end{pmatrix}
\qquad
b = 
\begin{pmatrix}
3\\2\\1\\3
\end{pmatrix}
\end{equation}

The solution of the system is
\begin{equation}
x= 
\begin{pmatrix}
-8/9\\0\\-1/9\\13/3
\end{pmatrix}
\end{equation}

In [201]:
A0 = np.array([[3,2,6],
               [2,0,5],
               [6,2,3]
               ],dtype=np.float64)
b0 = np.array([1,2,3], dtype=np.float64)
L,U,rowpiv = LUPP(A0)

# print("LU", L@U)
# print("PA",perm(rowpiv)@A0)
x_np = np.linalg.solve(A0,b0)

x_GEPP = GEPP(A0,b0)
x_GE = GE(A0,b0)

dferror = {'Errors:':['||x-xaprox||','||b-Axaprox||'], 'Gaussian Ellimination':[np.linalg.norm(x_np-x_GE), np.linalg.norm(b0-A0.dot(x_GE))], 'Gaussian Ellimination with PP':[np.linalg.norm(x_np-x_GEPP),  np.linalg.norm(b0-A0.dot(x_GEPP))]}
df = pd.DataFrame(data=dferror)
df
# print("Soluciones:\n")
# print(x_np )
# print(x_GE )
# print(x_GEPP )


[1. 2. 3.]
U:
 [[6.  2.  3. ]
 [0.  1.  4.5]
 [0.  0.  7. ]]
[2 2]
[2 2]


,Errors:,Gaussian Ellimination,Gaussian Ellimination with PP
0,||x-xaprox||,1.387779e-17,0.000000e+00
1,||b-Axaprox||,4.965068e-16,4.577567e-16


# Example. 
Consider the system $Ax = b$ with 
\begin{equation}
A = 
\begin{pmatrix}
1 & 3 & 4 & 1 \\ 2 & 1 & 5 & 1 \\ 4 & 1 & 6 & 1 \\ 6 & 2 & 3 & 2
\end{pmatrix}
\qquad
b = 
\begin{pmatrix}
3\\2\\1\\3
\end{pmatrix}
\end{equation}

The solution of the system is
\begin{equation}
x= 
\begin{pmatrix}
-8/9\\0\\-1/9\\13/3
\end{pmatrix}
\end{equation}

In [202]:
A0 = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
b0 = np.array([3,2,1,3], dtype=np.float64)
# L,U,rowpiv, colpiv = LUCP(A0)
# print(L@U)
# print(perm(rowpiv)@A0@perm(colpiv).T)

x_GECP = GECP(A0,b0)
x_GEPP = GEPP(A0,b0)
x_GE = GE(A0,b0)
x_np = np.linalg.solve(A0,b0)

print("Soluciones:\n")
print(x_np )
print(x_GE )
print(x_GEPP )
print(x_GECP )

U:
 [[6.         3.         1.         1.        ]
 [0.         4.5        1.5        1.5       ]
 [0.         0.         2.66666667 0.66666667]
 [0.         0.         0.         0.25      ]]
[2 3 2]
GECP [-1.11111111e-01 -8.88888889e-01 -3.33066907e-16  4.33333333e+00]
[2 2 2]
[3. 2. 1. 3.]
U:
 [[6.         2.         3.         2.        ]
 [0.         2.66666667 3.5        0.66666667]
 [0.         0.         4.5        0.        ]
 [0.         0.         0.         0.25      ]]
[3 3 2]
[3 3 2]
Soluciones:

[-0.88888889  0.         -0.11111111  4.33333333]
[-0.88888889 -0.         -0.11111111  4.33333333]
[-0.88888889  0.         -0.11111111  4.33333333]
[-8.88888889e-01 -3.33066907e-16 -1.11111111e-01  4.33333333e+00]


In [203]:

dferror = {'Errors:':['||x-xaprox||','||b-Axaprox||'], 
           'Gaussian Ellimination':[np.linalg.norm(x_np-x_GE), np.linalg.norm(b0-A0.dot(x_GE))], 
           'Gaussian Ellimination with PP':[np.linalg.norm(x_np-x_GEPP),  np.linalg.norm(b0-A0.dot(x_GEPP))],
           'Gaussian Ellimination with CP':[np.linalg.norm(x_np-x_GECP),  np.linalg.norm(b0-A0.dot(x_GECP))]}
df = pd.DataFrame(data=dferror)

df

,Errors:,Gaussian Ellimination,Gaussian Ellimination with PP,Gaussian Ellimination with CP
0,||x-xaprox||,4.652682e-16,1.110223e-16,9.501979e-16
1,||b-Axaprox||,2.843558e-15,9.930137e-16,1.935740e-15


In [209]:
e1 = []; e2 = []
N = [5*i for i in range(1,3)]
for nn in N:
    A_wc = -np.tril(np.ones((nn,nn)),-1)+np.eye(nn)
    A_wc[:,-1] = np.ones(nn)
    print(A_wc)
#     stop
#     x = np.ones(nn) 
    
    x = np.random.rand(nn); 
    b = A_wc.dot(x)
    
    x_gepp = Eliminacion_Gaussiana(A_wc,b, pivoteo='parcial')
    print("x     :\n", x)
    print("x_gepp:\n", x_gepp)
    e1.append(np.linalg.norm(x-x_gepp))
    e2.append(np.linalg.norm(b-A_wc.dot(x_gepp)))

dferror = {'n':N, '||x-x_gepp||':e1, '||b-A*x||':e2}
df = pd.DataFrame(data=dferror)
df

[[ 1.  0.  0.  0.  1.]
 [-1.  1.  0.  0.  1.]
 [-1. -1.  1.  0.  1.]
 [-1. -1. -1.  1.  1.]
 [-1. -1. -1. -1.  1.]]
[ 1.05074673 -0.08078724 -0.1339856  -1.54623909 -2.42907134]
U:
 [[ 1.  0.  0.  0.  1.]
 [ 0.  1.  0.  0.  2.]
 [ 0.  0.  1.  0.  4.]
 [ 0.  0.  0.  1.  8.]
 [ 0.  0.  0.  0. 16.]]
[0 1 2 3]
[0 1 2 3]
x     :
 [0.81077528 0.49001658 0.92683481 0.44141612 0.23997145]
x_gepp:
 [0.81077528 0.49001658 0.92683481 0.44141612 0.23997145]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [-1.  1.  0.  0.  0.  0.  0.  0.  0.  1.]
 [-1. -1.  1.  0.  0.  0.  0.  0.  0.  1.]
 [-1. -1. -1.  1.  0.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1. -1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1. -1. -1.  1.  0.  0.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.  0.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.  1.]]
[ 0.22410729  0.418814   -0.72989422 -0.25543559 -1.0281414  -0.88921348
 -1.65548289 -2.77585946 -3.50178721 -5.136448

,n,||x-x_gepp||,||b-A*x||
0,5,2.020636e-16,4.980560e-16
1,10,1.123463e-15,1.727994e-15


In [210]:
e1 = []; e2 = []
N = [5*i for i in range(1,3)]
for nn in N:
    A_wc = -np.tril(np.ones((nn,nn)),-1)+np.eye(nn)
    A_wc[:,-1] = np.ones(nn)
    xsol = np.ones(nn) # np.random.rand(nn); 
    b = A_wc.dot(xsol)
    
    x_gecp = Eliminacion_Gaussiana(A_wc,b, pivoteo='completo')
    print(A_wc.dot(x_gecp)-b)
    e1.append(np.linalg.norm(xsol-x_gecp))
    e2.append(np.linalg.norm(b-A_wc.dot(x_gecp)))

dferror = {'n':N, '||x-x_gecp||':e1, '||b-A*x_gecp||':e2}
df = pd.DataFrame(data=dferror)
df

U:
 [[ 1.  1.  0.  0.  0.]
 [ 0.  2.  0.  0.  1.]
 [ 0.  0.  1.  0. -2.]
 [ 0.  0.  0.  1. -4.]
 [ 0.  0.  0.  0. -8.]]
[0 1 2 3]
GECP [1. 1. 1. 1. 1.]
[0 4 2 3]
[0. 0. 0. 0. 0.]
U:
 [[   1.    1.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    2.    0.    0.    0.    0.    0.    0.    0.    1.]
 [   0.    0.    1.    0.    0.    0.    0.    0.    0.   -2.]
 [   0.    0.    0.    1.    0.    0.    0.    0.    0.   -4.]
 [   0.    0.    0.    0.    1.    0.    0.    0.    0.   -8.]
 [   0.    0.    0.    0.    0.    1.    0.    0.    0.  -16.]
 [   0.    0.    0.    0.    0.    0.    1.    0.    0.  -32.]
 [   0.    0.    0.    0.    0.    0.    0.    1.    0.  -64.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    1. -128.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0. -256.]]
[0 1 2 3 4 5 6 7 8]
GECP [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0 9 2 3 4 5 6 7 8]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


,n,||x-x_gecp||,||b-A*x_gecp||
0,5,0.0,0.0
1,10,0.0,0.0
